In [ ]:
import os
import pickle
import glob
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from typing import Dict, Any, List

# --- 設定と定数 ---
# pickleファイルが保存されているフォルダのパス
# 実行する前に、このフォルダ内に分析結果のpickleファイルが存在することを確認してください。
OUTPUT_FOLDER = "sliding_window_results"
# 出力するCSVファイル名
CSV_FILENAME = "model_summary_results.csv"

# 既存のモデルデータを読み込み、集計、可視化する機能のみを残します。
# データのシミュレーション関数は削除しました。


def process_and_visualize_models(folder_path: str, csv_filename: str):
    """
    複数のpickleファイルからモデルの結果を読み込み、DataFrame化、可視化、CSV出力を行うメイン関数。

    【重要】この関数は、期間（ウィンドウ）ごとに変数の数が異なっても柔軟に対応します。

    各pickleファイルは以下の形式の辞書を保存していることを前提とします:
    {
        'window_index': int,
        'coefficients': {'var1': float, 'var2': float, ...},  # 推定量β (キーセットはウィンドウごとに異なっても良い)
        'variance': {'var1': float, 'var2': float, ...}      # 推定量の分散 (キーセットはウィンドウごとに異なっても良い)
    }
    """

    # 1. ファイルの検索と初期設定
    search_path = os.path.join(folder_path, "*.pkl")
    file_list = sorted(glob.glob(search_path))

    if not file_list:
        print(f"エラー: フォルダ {folder_path} 内にpickleファイルが見つかりませんでした。")
        print("指定されたフォルダパスが正しいか、ファイル名が '*.pkl' であるか確認してください。")
        return

    print(f"読み込むファイル数: {len(file_list)}")

    # 全結果を格納するためのリスト
    all_results: List[Dict[str, Any]] = []

    # 2. データの読み込みと抽出
    for filepath in file_list:
        try:
            with open(filepath, 'rb') as f:
                data = pickle.load(f)

            # 各ファイルのデータ構造が想定通りであることを確認
            window_index = data.get('window_index')
            coefficients = data.get('coefficients')
            variance = data.get('variance')

            if window_index is None or coefficients is None or variance is None:
                print(f"警告: ファイル {os.path.basename(filepath)} のデータ構造が不正です（必要なキー(window_index, coefficients, variance)が不足）。スキップします。")
                continue

            # 各変数/推定量に対してデータを整形
            # coefficients.items()を使用することで、ウィンドウ内の変数セットのみを処理し、
            # 次のステップでDataFrameに変換される際に、全てのウィンドウの変数が結合され、
            # 存在しない場所にはNaNが自動的に割り当てられます。
            for var_name, weight in coefficients.items():
                var_variance = variance.get(var_name)

                # 必要な情報が揃っているか確認
                if var_variance is not None:
                    all_results.append({
                        'window_index': window_index,
                        'variable_name': var_name,
                        'weight_beta': weight,  # 推定量β (係数)
                        'variance_beta': var_variance, # 推定量の分散
                        'std_err_beta': np.sqrt(var_variance) # 標準誤差 (標準偏差)
                    })
                else:
                    # 分散が欠落している場合は、その変数の係数もスキップ（または分散にNaNを入れても良いが、今回はスキップ）
                    print(f"警告: ファイル {os.path.basename(filepath)} の変数 {var_name} の分散データが見つかりませんでした。スキップします。")


        except Exception as e:
            print(f"致命的なエラー: ファイル {os.path.basename(filepath)} の読み込み中に問題が発生しました: {e}")
            continue

    # 3. DataFrameの作成
    if not all_results:
        print("エラー: 有効なデータが抽出されませんでした。処理を終了します。")
        return

    # ここで異なるウィンドウの変数が結合され、自動的に欠損値(NaN)が処理されます。
    df = pd.DataFrame(all_results)

    # NaNが含まれる場合、可視化ライブラリが適切に処理するため、特別な前処理は不要です。
    print("\n--- データフレームの概要 ---")
    print(df.head())
    print(df.info())

    # 4. 可視化 (seabornとmatplotlibを使用)
    sns.set_theme(style="whitegrid")

    # Figure 1: 推定量β (係数) の時系列変化
    # 変数が欠損しているウィンドウでは、seabornは自動的にその点をプロットしません。
    plt.figure(figsize=(14, 6))
    sns.lineplot(
        data=df,
        x='window_index',
        y='weight_beta',
        hue='variable_name',
        marker='o'
    )
    plt.axhline(0, color='grey', linestyle='--', linewidth=1) # 0のラインを追加
    plt.title('スライディングウィンドウによる推定量 β (係数) の推移', fontsize=16)
    plt.xlabel('ウィンドウ番号', fontsize=12)
    plt.ylabel('推定量 β (係数)', fontsize=12)
    plt.legend(title='変数名', bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.tight_layout()
    plt.show()

    # Figure 2: 分散 (信頼性の指標) の時系列変化
    plt.figure(figsize=(14, 6))
    sns.lineplot(
        data=df,
        x='window_index',
        y='variance_beta',
        hue='variable_name',
        marker='s'
    )
    plt.title('スライディングウィンドウによる推定量 β の分散の推移', fontsize=16)
    plt.xlabel('ウィンドウ番号', fontsize=12)
    plt.ylabel('分散', fontsize=12)
    plt.legend(title='変数名', bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.tight_layout()
    plt.show()

    # 5. CSVファイルへの出力
    output_filepath = os.path.join(os.getcwd(), csv_filename)
    df.to_csv(output_filepath, index=False, encoding='utf-8')
    print(f"\n--- 処理完了 ---")
    print(f"集計結果はCSVファイルに保存されました: {output_filepath}")

# --- スクリプトの実行 ---
if __name__ == "__main__":
    # 既存のデータ処理を実行
    process_and_visualize_models(OUTPUT_FOLDER, CSV_FILENAME)